<a href="https://colab.research.google.com/github/mursaleen-sengr/Fake-news-Classification/blob/main/FakeNewsClassifierUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fake News Classifier Using LSTM

Dataset used in this project: https://www.kaggle.com/c/fake-news/data#

In [1]:
#importing libraries
import pandas as pd
import numpy as np

import tensorflow as tf

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


import nltk
import re
from nltk.corpus import stopwords



In [2]:
#reading the dataset
df=pd.read_csv('/content/train.csv')

In [3]:
#data set exploration
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
#checking shape [rows and columns ]
df.shape

(20800, 5)

In [5]:
# checking and calculating  the null values
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
# removing the null  Values
df=df.dropna()


In [7]:
df.shape

(18285, 5)

In [8]:
#getting the Independent Features out of dataset

X=df.drop('label',axis=1)

In [9]:
## Get the Dependent features
y=df['label']

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
#checking the  version  of tensorflow
tf.__version__

'2.15.0'

In [13]:
# Vocabulary size  this can be random because the dataset is large I will be taking 5000 so it takes lesser time to execute
voc_size=5000  # 5k unique words

### Onehot Representation

In [14]:
# creating the copy of dataset without independent feature because the experimentation doesnt make any changes in original dataset
messages=X.copy()

In [16]:
#working with title column not with text because of resource limitation
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [17]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [18]:
messages.reset_index(inplace=True)

In [19]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [20]:
# removing the stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
# Data Preprocessing
from nltk.stem.porter import PorterStemmer # for stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower() #converting to lower case
    review = review.split() # spliting means getting words by words

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [ ]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [23]:
#one hot representation
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[4429, 4870, 2130, 4174, 349, 3402, 1946, 1333, 1877, 2491],
 [2124, 2209, 2910, 17, 2517, 4227, 1482],
 [2058, 4054, 2121, 4309],
 [802, 541, 2120, 215, 3575, 3849],
 [1892, 2517, 1486, 3343, 365, 4698, 2517, 2727, 4584, 3746],
 [2691,
  2000,
  1647,
  1516,
  329,
  223,
  3484,
  979,
  239,
  1475,
  1361,
  4721,
  4940,
  421,
  1482],
 [3183, 2686, 1552, 1333, 2790, 162, 3271, 4327, 2612, 754, 3493],
 [2857, 1400, 4623, 1550, 2047, 4386, 223, 602, 2612, 754, 3493],
 [4052, 4947, 372, 2360, 3399, 3225, 3817, 4891, 223, 288],
 [796, 3200, 3546, 4947, 3540, 897, 1885, 4860],
 [317, 2414, 3042, 2964, 2557, 1183, 4284, 2510, 4418, 4210, 2234],
 [215, 1017, 349, 3225, 223, 2047],
 [732, 145, 476, 2501, 4982, 3915, 667, 3164, 546],
 [1643, 3776, 4099, 4944, 3045, 2342, 2731, 2612, 754, 3493],
 [3292, 1538, 1624, 2606, 183, 2612, 754, 3493],
 [4209, 3477, 3910, 738, 2086, 1039, 1241, 1367, 55, 3620],
 [473, 2075, 2209],
 [1541, 1278, 1222, 3762, 223, 1393, 4904, 1482],
 [2547, 2820, 2

In [24]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [25]:
onehot_repr[1]

[2124, 2209, 2910, 17, 2517, 4227, 1482]

### Embedding Representation

In [29]:
sent_length=20 # maximum length of a sentence
#each sentence length will be of 20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length) #using post padding techniques to make the size equal
print(embedded_docs)

[[4429 4870 2130 ...    0    0    0]
 [2124 2209 2910 ...    0    0    0]
 [2058 4054 2121 ...    0    0    0]
 ...
 [1807 1859 3415 ...    0    0    0]
 [1697 2047 3784 ...    0    0    0]
 [2986 2768 1631 ...    0    0    0]]


In [30]:
embedded_docs[1]

array([2124, 2209, 2910,   17, 2517, 4227, 1482,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [31]:
embedded_docs[0]

array([4429, 4870, 2130, 4174,  349, 3402, 1946, 1333, 1877, 2491,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

## Training the Model

In [32]:
## Creating model
embedding_vector_features=40 #features representation will of 40 size
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid')) # because of binary classification using sigmoid activation function
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [59]:
len(embedded_docs),y.shape

(18285, (18285,))

In [35]:

X_final=np.array(embedded_docs)
y_final=np.array(y)

In [36]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [58]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 13s 50ms/step - loss: 0.3306 - accuracy: 0.8306 - val_loss: 0.2041 - val_accuracy: 0.9155
Epoch 2/10
192/192 [==============================] - 2s 12ms/step - loss: 0.1422 - accuracy: 0.9451 - val_loss: 0.2062 - val_accuracy: 0.9190
Epoch 3/10
192/192 [==============================] - 2s 10ms/step - loss: 0.1027 - accuracy: 0.9622 - val_loss: 0.2373 - val_accuracy: 0.9170
Epoch 4/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0884 - accuracy: 0.9677 - val_loss: 0.2891 - val_accuracy: 0.9125
Epoch 5/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0684 - accuracy: 0.9765 - val_loss: 0.3233 - val_accuracy: 0.9112
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0527 - accuracy: 0.9829 - val_loss: 0.2920 - val_accuracy: 0.9115
Epoch 7/10
192/192 [==============================] - 2s 13ms/step - loss: 0.0452 - accuracy: 0.9851 - val_loss: 0.3401 - val_accuracy: 0.9094

# Model Evaluation

In [65]:
y_pred=model.predict(X_test)

189/189 [==============================] - 0s 2ms/step


In [66]:
y_pred=np.where(y_pred > 0.6, 1,0)

In [67]:
from sklearn.metrics import confusion_matrix

In [68]:
confusion_matrix(y_test,y_pred)

array([[3156,  263],
       [ 340, 2276]])

In [69]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9000828500414251

In [70]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91      3419
           1       0.90      0.87      0.88      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

